In [1]:
import numpy as np
x = np.linspace(-10,10,5000)
deltax = x[1]-x[0]

In [2]:
def norm(phi):
    norm = np.sum(np.square(np.abs(phi)))*deltax
    return phi/np.sqrt(norm)


def complex_plot(x,y):
    real = np.real(y)
    imag = np.imag(y)
    a,*_ = plt.plot(x,real, ':', label='Re')
    b,*_ = plt.plot(x,imag, ':', label='Im')
    plt.xlim(-2,2)
    p,*_ = plt.plot(x,np.abs(y),label='$\sqrt{P}$')
    return a,b,p
    
def wave_packet(pos=0,mom=0,sigma=0.2):
    return norm(np.exp(-1j*mom*x)*np.exp(-np.square(x-pos)/sigma/sigma,dtype=complex))



def d_dxdx(phi,x=x):
    dphi_dxdx = -2*phi
    dphi_dxdx[:-1] += phi[1:]
    dphi_dxdx[1:] += phi[:-1]
    return dphi_dxdx/deltax

def d_dt(phi,h=1,m=100,V=0):
    return 1j*h/2/m * d_dxdx(phi) - 1j*V*phi/h


def rk4(phi, dt, **kwargs):
    k1 = d_dt(phi, **kwargs)
    k2 = d_dt(phi+dt/2*k1, **kwargs)
    k3 = d_dt(phi+dt/2*k2, **kwargs)
    k4 = d_dt(phi+dt*k3, **kwargs)
    return phi + dt/6*(k1+2*k2+2*k3+k4)


def simulate(phi_sim, 
             V=0, 
             steps=100000, 
             dt=1e-1, 
             condition=None, 
             save_every=1000):
    simulation_steps = np.zeros((int(steps/save_every), 5000), dtype = complex)
    l = 0
    for i in range(steps):
        phi_sim = rk4(phi_sim,dt,V=V)
#         if condition:
#             phi_sim = condition(phi_sim)
        phi_sim = norm(phi_sim)
        if i % save_every == 0:
            simulation_steps[l,:] = phi_sim
            l += 1
    return simulation_steps

In [3]:
# sim_free = simulate(wave_packet(mom=-5),steps=100000,save_every=1000)

In [4]:
# print((sim_free))

In [5]:
%matplotlib tk

from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

def animate(simulation_steps,init_func=None):
    fig = plt.figure()
    re,im,prob = complex_plot(x,simulation_steps[0])
    plt.xlim(-2,2)
    plt.ylim(-2,2)
    if init_func:
        init_func()
    plt.legend()
    
    def box_init():
        plt.gcf().axes[0].axvspan(1, 1.2, alpha=0.2, color='orange')
        plt.xlim(-2,4)
        plt.ylim(-3,3)

    def animate(frame):
        prob.set_data((x, np.abs(simulation_steps[frame])))
        re.set_data((x, np.real(simulation_steps[frame])))
        im.set_data((x, np.imag(simulation_steps[frame])))
        return prob,re,im

    anim = FuncAnimation(fig, animate, frames=int(len(simulation_steps)), init_func = box_init(), interval=50)
    plt.show()

    return anim

In [6]:
# animate(sim_free)

In [7]:
box_potential = np.where((x>1)&(x<1.2),0.02,0)
sim_box_mom = simulate(wave_packet(mom=-20),V=box_potential,steps=50000,save_every=1000)

In [8]:
animate(sim_box_mom)

In [9]:
prob_before_barrier = np.sum(np.square(np.abs(sim_box_mom[-1,:])), where=(x>1.2))*deltax

0.005660456997527719

In [10]:
prob_after_barrier = np.sum(np.square(np.abs(sim_box_mom[-1,:])), where=(x<1.2))*deltax

0.9943395430024725